# 03 — Walk-Forward Validation (Rolling Cointegration + Out-of-Sample Backtest)

This notebook implements the **“Next improvements”** section from the README:

- **Walk-forward validation** with **rolling cointegration tests**
- **Out-of-sample (OOS)** trading simulation for selected pairs
- (Optional hooks) dynamic hedge ratio, regime filters, transaction costs

> **Why this notebook exists:** Cointegration at a single point in time can look “significant,” yet fail in live trading because relationships drift. Walk-forward validation tests whether a pair remains tradable **out of sample**, not just in-sample.

---

## What you’ll get

1. **Rolling windows** (train → trade) across time  
2. For each window:
   - Select pair(s) by Engle–Granger p-value on the **train** period
   - Estimate hedge ratio (**beta**) on train (OLS)
   - Compute spread + z-score using train statistics
   - Trade **only** in the **next OOS** period
3. Aggregate OOS equity curve + performance metrics.

---

## Assumptions / Conventions

- Price input: **Adjusted Close**
- Spread: \( s_t = p^A_t - \beta p^B_t \)
- Z-score uses **train mean/std** (or rolling in trade, configurable)
- Entry/exit thresholds default to the README: entry \(|z|>2\), exit \(|z|<0.5\)
- Position sizing: 1x notional per leg (dollar-neutral), single pair at a time (configurable)

---

## Data caching (no parquet)

This repo previously hit a parquet ImportError when `pyarrow/fastparquet` aren’t installed.  
This notebook uses **CSV caching** by default.


In [ ]:
# If you're running from the repo root, paths will just work.
# If not, adjust PROJECT_ROOT accordingly.

from __future__ import annotations

import os
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

warnings.filterwarnings("ignore")

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
CACHE_DIR = DATA_DIR / "cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CACHE_DIR:", CACHE_DIR)


In [ ]:
# --- Data Loading ---
# This notebook supports:
# 1) Pulling prices from yfinance (most convenient)
# 2) Loading an existing CSV cache

TICKERS = ["NVDA", "JPM", "AMZN", "META"]  # update freely
START = "2016-01-01"
END = None  # None = today

PRICE_CACHE_CSV = CACHE_DIR / f"adj_close_{START}_to_{END or 'today'}_{'-'.join(TICKERS)}.csv"

def download_adj_close_yf(tickers: list[str], start: str, end: str | None) -> pd.DataFrame:
    import yfinance as yf
    df = yf.download(
        tickers,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False
    )
    # yfinance returns columns like ('Adj Close', 'NVDA') if multiple tickers
    if isinstance(df.columns, pd.MultiIndex):
        out = df["Adj Close"].copy()
    else:
        out = df.rename(columns={"Adj Close": tickers[0]})[tickers[0]].to_frame()
    out.index = pd.to_datetime(out.index)
    out = out.sort_index()
    out = out.dropna(how="all")
    return out

def load_prices(tickers: list[str], start: str, end: str | None, cache_path: Path) -> pd.DataFrame:
    if cache_path.exists():
        prices = pd.read_csv(cache_path, index_col=0, parse_dates=True)
        # keep only requested tickers if cache has more
        missing = [t for t in tickers if t not in prices.columns]
        if missing:
            raise ValueError(f"Cache exists but missing tickers: {missing}. Delete cache or update tickers.")
        return prices[tickers].copy()
    prices = download_adj_close_yf(tickers, start, end)
    prices.to_csv(cache_path)
    print(f"Saved cache -> {cache_path}")
    return prices

prices = load_prices(TICKERS, START, END, PRICE_CACHE_CSV)
prices = prices.dropna()
prices.head()


In [ ]:
# Quick sanity checks
prices.describe().T[["count", "mean", "std", "min", "max"]]


In [ ]:
# Plot prices (log scale helps for long horizons)
ax = (np.log(prices)).plot(figsize=(12, 5), title="Log Adj Close")
ax.set_xlabel("Date")
plt.show()


In [ ]:
# --- Helpers: cointegration, hedge ratio, z-score, backtest ---

def ols_beta(y: pd.Series, x: pd.Series) -> float:
    """Estimate beta from y ~ alpha + beta*x via OLS."""
    df = pd.concat([y, x], axis=1).dropna()
    yv = df.iloc[:, 0].values
    xv = sm.add_constant(df.iloc[:, 1].values)
    model = sm.OLS(yv, xv).fit()
    return float(model.params[1])

def spread_series(pA: pd.Series, pB: pd.Series, beta: float) -> pd.Series:
    return pA - beta * pB

def zscore_from_train(spread: pd.Series, train_mask: pd.Series, eps: float = 1e-12) -> pd.Series:
    mu = spread[train_mask].mean()
    sd = spread[train_mask].std(ddof=0)
    sd = max(sd, eps)
    return (spread - mu) / sd

def coint_pvalue(y: pd.Series, x: pd.Series) -> float:
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < 60:
        return np.nan
    score, pvalue, _ = coint(df.iloc[:, 0], df.iloc[:, 1])
    return float(pvalue)

def adf_pvalue(s: pd.Series) -> float:
    s = s.dropna()
    if len(s) < 60:
        return np.nan
    res = adfuller(s, autolag="AIC")
    return float(res[1])

def backtest_pair_oos(
    pA: pd.Series,
    pB: pd.Series,
    train_mask: pd.Series,
    trade_mask: pd.Series,
    entry_z: float = 2.0,
    exit_z: float = 0.5,
    cost_bps: float = 0.0,
) -> pd.DataFrame:
    """Dollar-neutral backtest trading only during trade_mask, signals calibrated on train_mask.

    Positions:
      - If z > entry: short spread (short A, long beta*B)
      - If z < -entry: long spread (long A, short beta*B)
      - Exit when |z| < exit_z
    Transaction cost: 'cost_bps' per *gross* notional turn (simple approximation).
    """
    # Estimate beta on train
    beta = ols_beta(pA[train_mask], pB[train_mask])

    spr = spread_series(pA, pB, beta).rename("spread")
    z = zscore_from_train(spr, train_mask).rename("z")

    idx = spr.index
    pos = pd.Series(0.0, index=idx)  # +1 long spread, -1 short spread
    state = 0.0

    # only generate trades inside trade period; stay flat outside
    for t in idx:
        if not trade_mask.loc[t]:
            state = 0.0
            pos.loc[t] = 0.0
            continue

        zt = z.loc[t]
        if state == 0.0:
            if zt <= -entry_z:
                state = +1.0
            elif zt >= entry_z:
                state = -1.0
        else:
            # exit rule
            if abs(zt) <= exit_z:
                state = 0.0
        pos.loc[t] = state

    # returns: approximate dollar-neutral PnL using price changes
    # spread return proxy: Δ(A - beta*B)
    ds = spr.diff().fillna(0.0)

    pnl = pos.shift(1).fillna(0.0) * ds  # enter at next bar
    pnl = pnl.rename("pnl")

    # simple costs: whenever position changes, pay cost_bps on gross notional
    turnover = pos.diff().abs().fillna(0.0).rename("turnover")
    # Gross notional ~ 1 + |beta| (one unit A plus beta units of B); scale cost accordingly.
    gross = (1.0 + abs(beta))
    cost = (turnover * (cost_bps / 10_000.0) * gross).rename("cost")

    net = (pnl - cost).rename("net_pnl")

    out = pd.concat([spr, z, pos.rename("position"), pnl, cost, net], axis=1)
    out["beta"] = beta
    out["train_mean_spread"] = spr[train_mask].mean()
    out["train_std_spread"] = spr[train_mask].std(ddof=0)
    out["is_train"] = train_mask.astype(int)
    out["is_trade"] = trade_mask.astype(int)
    return out

def perf_stats(equity: pd.Series, freq: int = 252) -> dict:
    r = equity.pct_change().dropna()
    ann_ret = (equity.iloc[-1] ** (freq / max(len(r), 1)) - 1.0) if len(r) else np.nan
    ann_vol = r.std(ddof=0) * np.sqrt(freq) if len(r) else np.nan
    sharpe = (r.mean() / (r.std(ddof=0) + 1e-12)) * np.sqrt(freq) if len(r) else np.nan

    # max drawdown
    peak = equity.cummax()
    dd = (equity / peak) - 1.0
    max_dd = dd.min() if len(dd) else np.nan

    win_rate = (r > 0).mean() if len(r) else np.nan

    return {
        "annualized_return": float(ann_ret) if np.isfinite(ann_ret) else np.nan,
        "annualized_vol": float(ann_vol) if np.isfinite(ann_vol) else np.nan,
        "sharpe": float(sharpe) if np.isfinite(sharpe) else np.nan,
        "max_drawdown": float(max_dd) if np.isfinite(max_dd) else np.nan,
        "win_rate_daily": float(win_rate) if np.isfinite(win_rate) else np.nan,
        "equity_start": float(equity.iloc[0]) if len(equity) else np.nan,
        "equity_end": float(equity.iloc[-1]) if len(equity) else np.nan,
        "n_days": int(len(equity)),
    }


## Walk-forward design

We split the timeline into repeated **train → trade** windows:

- `train_window_days`: length used for **pair selection** and calibration (beta + spread mean/std)
- `trade_window_days`: subsequent OOS window where we **actually trade**
- `step_days`: how much we slide forward each iteration

Typical choices:
- train: 252–756 trading days
- trade: 63–252 trading days
- step: trade_window_days (non-overlapping) or smaller (overlapping)

Below we implement a flexible walk-forward runner.


In [ ]:
from itertools import combinations

# --- Walk-forward configuration ---
TRAIN_WINDOW_DAYS = 504   # ~2 years
TRADE_WINDOW_DAYS = 126   # ~6 months
STEP_DAYS = 126           # slide by 6 months

ENTRY_Z = 2.0
EXIT_Z = 0.5
COST_BPS = 0.0  # set e.g., 1-5 bps if you want a simple friction term

# pair selection criteria on TRAIN window
MAX_COINTEGRATION_P = 0.05
TOP_K_PAIRS_PER_WINDOW = 1  # trade best pair per window (ranked by p-value)

all_pairs = list(combinations(prices.columns.tolist(), 2))
all_pairs


In [ ]:
def iter_windows(index: pd.DatetimeIndex, train_days: int, trade_days: int, step_days: int):
    # Use integer positions (trading days) rather than calendar days
    n = len(index)
    start = 0
    while True:
        train_start = start
        train_end = train_start + train_days
        trade_end = train_end + trade_days
        if trade_end > n:
            break
        yield (train_start, train_end, trade_end)
        start += step_days

def select_pairs_train(prices_slice: pd.DataFrame, pairs: list[tuple[str, str]], max_p: float, top_k: int):
    rows = []
    for a, b in pairs:
        p = coint_pvalue(prices_slice[a], prices_slice[b])
        rows.append((a, b, p))
    sel = pd.DataFrame(rows, columns=["asset_A", "asset_B", "coint_pvalue"]).dropna()
    sel = sel.sort_values("coint_pvalue", ascending=True)
    sel = sel[sel["coint_pvalue"] <= max_p].head(top_k)
    return sel.reset_index(drop=True)

# Run walk-forward
windows = list(iter_windows(prices.index, TRAIN_WINDOW_DAYS, TRADE_WINDOW_DAYS, STEP_DAYS))
print(f"Num windows: {len(windows)}")

window_summaries = []
oos_trades = []  # store detailed trade frames per window

for w, (i0, i1, i2) in enumerate(windows, start=1):
    idx = prices.index
    train_idx = idx[i0:i1]
    trade_idx = idx[i1:i2]

    train_mask = prices.index.isin(train_idx)
    trade_mask = prices.index.isin(trade_idx)

    train_prices = prices.loc[train_idx]

    selected = select_pairs_train(train_prices, all_pairs, MAX_COINTEGRATION_P, TOP_K_PAIRS_PER_WINDOW)
    if selected.empty:
        window_summaries.append({
            "window": w,
            "train_start": train_idx[0],
            "train_end": train_idx[-1],
            "trade_start": trade_idx[0],
            "trade_end": trade_idx[-1],
            "selected_pairs": 0,
            "note": "No pairs passed cointegration threshold"
        })
        continue

    # Trade each selected pair, then combine (here: equal-weight if >1)
    pair_equities = []
    pair_meta = []

    for _, row in selected.iterrows():
        a, b = row["asset_A"], row["asset_B"]
        bt = backtest_pair_oos(
            prices[a],
            prices[b],
            train_mask=pd.Series(train_mask, index=prices.index),
            trade_mask=pd.Series(trade_mask, index=prices.index),
            entry_z=ENTRY_Z,
            exit_z=EXIT_Z,
            cost_bps=COST_BPS,
        )

        # Equity during trade period only; start at 1.0 per window/pair
        trade_bt = bt.loc[trade_idx].copy()
        eq = (1.0 + trade_bt["net_pnl"]).cumprod()
        eq.name = f"equity_{a}_{b}"
        pair_equities.append(eq)

        # Add diagnostics
        spr_adf_p = adf_pvalue(bt.loc[train_idx, "spread"])
        pair_meta.append({
            "asset_A": a,
            "asset_B": b,
            "coint_pvalue_train": float(row["coint_pvalue"]),
            "adf_pvalue_spread_train": spr_adf_p,
            "beta_train": float(bt["beta"].iloc[0]),
        })

        # keep full detail for debugging
        bt["window"] = w
        bt["asset_A"] = a
        bt["asset_B"] = b
        oos_trades.append(bt)

    # Combine if multiple pairs (equal weight on equity curves)
    equity_mat = pd.concat(pair_equities, axis=1).fillna(method="ffill").fillna(1.0)
    # average equity across pairs (rough equal-weight)
    combined = equity_mat.mean(axis=1).rename("equity_window")

    stats = perf_stats(combined)
    window_summaries.append({
        "window": w,
        "train_start": train_idx[0],
        "train_end": train_idx[-1],
        "trade_start": trade_idx[0],
        "trade_end": trade_idx[-1],
        "selected_pairs": len(pair_meta),
        "pairs": pair_meta,
        **stats,
    })

window_summary_df = pd.DataFrame(window_summaries)
window_summary_df.head(10)


In [ ]:
# How often did we find a cointegrated pair in the training window?
window_summary_df["selected_pairs"].value_counts(dropna=False)


In [ ]:
# --- Build overall OOS equity curve by chaining window returns ---
# For windows with no selected pair, equity stays flat in that period.

idx = prices.index
overall = pd.Series(1.0, index=idx, name="equity_oos")

# We'll fill per-trade-window equity from the combined per-window result.
# To do that, we rebuild per-window combined equity from stored oos_trades.
oos_trades_df = pd.concat(oos_trades, axis=0) if oos_trades else pd.DataFrame()
oos_trades_df.shape


In [ ]:
def window_equity_from_trades(trades: pd.DataFrame, trade_idx: pd.DatetimeIndex) -> pd.Series:
    # If multiple pairs in a window, average their equity curves
    eqs = []
    for (a, b), df in trades.groupby(["asset_A", "asset_B"]):
        d = df.loc[trade_idx].copy()
        eq = (1.0 + d["net_pnl"]).cumprod()
        eqs.append(eq.rename(f"{a}_{b}"))
    if not eqs:
        return pd.Series(1.0, index=trade_idx, name="equity_window")
    mat = pd.concat(eqs, axis=1).fillna(method="ffill").fillna(1.0)
    return mat.mean(axis=1).rename("equity_window")

overall = pd.Series(1.0, index=prices.index, name="equity_oos")

if not oos_trades_df.empty:
    for w, (i0, i1, i2) in enumerate(windows, start=1):
        trade_idx = prices.index[i1:i2]
        w_trades = oos_trades_df[oos_trades_df["window"] == w]
        eq_w = window_equity_from_trades(w_trades, trade_idx)
        # chain multiplicatively from last value before trade window
        prev = overall.loc[trade_idx[0] - pd.Timedelta(days=1)] if trade_idx[0] - pd.Timedelta(days=1) in overall.index else overall.loc[:trade_idx[0]].iloc[-1]
        # normalize window equity to start at 1.0 then chain
        overall.loc[trade_idx] = prev * (eq_w / eq_w.iloc[0])

overall = overall.ffill()
overall.head(), overall.tail()


In [ ]:
# Plot OOS equity curve
ax = overall.plot(figsize=(12, 5), title="Walk-Forward OOS Equity Curve")
ax.set_xlabel("Date")
ax.set_ylabel("Equity")
plt.show()

# Drawdown
peak = overall.cummax()
dd = overall / peak - 1.0
ax = dd.plot(figsize=(12, 4), title="Drawdown (OOS)")
ax.set_xlabel("Date")
ax.set_ylabel("Drawdown")
plt.show()


In [ ]:
# Portfolio-level OOS metrics
oos_stats = perf_stats(overall)
pd.DataFrame([oos_stats]).T.rename(columns={0: "value"})


In [ ]:
# Window-by-window diagnostics (sort by Sharpe)
cols = [
    "window","train_start","train_end","trade_start","trade_end","selected_pairs",
    "annualized_return","sharpe","max_drawdown","win_rate_daily","equity_end"
]
window_summary_df[cols].sort_values("sharpe", ascending=False).head(15)


In [ ]:
# --- Export results ---
OUT_DIR = PROJECT_ROOT / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

window_summary_path = OUT_DIR / "walkforward_window_summary.csv"
equity_path = OUT_DIR / "walkforward_oos_equity.csv"
trades_path = OUT_DIR / "walkforward_oos_trades.csv"

window_summary_df.to_csv(window_summary_path, index=False)
overall.to_frame().to_csv(equity_path, index=True)

if not oos_trades_df.empty:
    # keep only trade rows to reduce file size
    oos_trade_rows = oos_trades_df[oos_trades_df["is_trade"] == 1].copy()
    oos_trade_rows.to_csv(trades_path, index=True)

print("Saved:")
print("-", window_summary_path)
print("-", equity_path)
print("-", trades_path if not oos_trades_df.empty else "(no trades to save)")


## Extensions (Optional)

If you want to go beyond the README improvements:

### 1) Rolling/Adaptive hedge ratio
- Replace static OLS beta with:
  - Rolling OLS (re-estimate beta every N days using last M days)
  - Kalman filter (time-varying beta)

### 2) Regime filters
- Volatility filter: trade only when realized vol is below a threshold
- Trend filter: avoid mean-reversion bets when the spread is trending

### 3) Transaction costs + slippage
- Model per-share costs, half-spread, and market impact
- Costs matter a lot for high-turnover strategies

### 4) Portfolio constraints
- Multiple pairs concurrently
- Gross/net exposure caps
- Risk parity sizing

If you want, I can add any of the above as extra sections in this notebook
(or as **04_dynamic_hedge_ratio_kalman.ipynb**).
